# Bias-Variance

... on going ...

by reading a few papers [cite] the methodology is as follows...

The Loss of a Algorithm (not of a model!) can be decomposed in:
$$L(A) = Bias(A) + Variance(A) + Noise(A)$$
where:
 - Bias is how far is the model from the prediction
 - Variance is how sensitive (how changes) the prediction with different training sets (overfitting)
 - Noise is the irreducible error in the dataset (learner independent)

Given a dataset $D$, given a number of iteration $l$, we need to produce $l$ score predictions
for each element in $D$ with $l$ different models. If the loss function $L$ is MSE then everything is clear.
If the loss function is NDCG, no previous work.

  0. Shuffle $D$ and create two halves $D_1$ and $D_2$ (this should be query-aware with NDCG)
  0. learn a model from $D_1$ and apply it to $D_2$
  0. learn a model from $D_2$ and apply it to $D_1$
  0. store the scores produced so far
  0. Repeat all of the above $l$ times
  0. for each document/query compute the average prediction by averagin the $l$ stored so far (for ranking this is actually not defined, we may average predictions and then rank accordingly)
  0. Bias of a query/document is computed as the loss between the ideal prediction and the average prediction
  0. Average above for every query/document to obtain final Bias
  0. Variance of a query/document is computed as the average loss between the average prediction and every of the $l$ predictions
  0. Average above for every query/document to obtain final Variance
 
Usually Noise is assumed to be 0 as it is impossible to estimated it.
As a consequence, ometimes Variance is defined as $L(A)-Bias(A)$.

In every work, $L(A)$ should be equal to $Bias(A) + Variance(A)$. Is this true for NDCG? probably not ...

In principle, we may provide just the Loss and the Bias, and let the user compute variance by difference.

We might probalby solve the ranking problem, by saying that L is not 1-NDCG, but rather (1-NDCG)^2

In [103]:
from rankeval.core.dataset import Dataset
import numpy as np
import lightgbm
from __future__ import print_function

%matplotlib inline
from matplotlib import pyplot as plt

In [157]:
dataset = Dataset.load("/Users/claudio/docs/LAVORO/coding/ltr/QuickScorer/debug_data/msn1.fold1.test.5k.txt", name="msn1")

In [158]:
def lgbm_algo(train_X, train_Y, train_q, test_X):
    params = {'num_leaves': 2, 'objective':'lambdarank',
             'learning_rate': 0.01, 'max_bin': 1024}

    training = lightgbm.Dataset(data=train_X, label=train_Y, group=train_q)
    
    bst = lightgbm.train(params, training, num_boost_round=100)
    
    # check the number of trees and other params is correct
    # print ( len(bst.dump_model()[u'tree_info']) )
    
    return bst.predict(test_X)


def kfold_scoring(dataset, k, algo):
    scores = np.zeros(dataset.n_instances, dtype=np.float32)
    q_lens = dataset.query_ids[1:]-dataset.query_ids[:-1]
    partition_ids = np.random.randint(k, size=dataset.n_queries)
    for p in range(k):
        print (" - Processing fold", p, "of", k)
        test_rows = np.zeros(dataset.n_instances, dtype=np.bool)
        for q in np.where(partition_ids==p)[0]:
            test_rows[ dataset.query_ids[q]:dataset.query_ids[q+1] ] = True
        train_rows = np.logical_not(test_rows)
        
        fold_scores = algo(dataset.X[train_rows], dataset.y[train_rows], q_lens[partition_ids!=p],
                           dataset.X[test_rows])
        
        scores[test_rows] = fold_scores
        
    return scores
            


In [162]:
L = 5
scores = np.empty( (dataset.n_instances, L), dtype=np.float32)
for l in range(5):
    print (" + Dataset scoring", l, "of", L)
    scores[:,l] = kfold_scoring(dataset, 2, algo=lgbm_algo)

print (scores.sum())

 + Dataset scoring 0 of 5
 - Processing fold 0 of 2
 - Processing fold 1 of 2
 + Dataset scoring 1 of 5
 - Processing fold 0 of 2
 - Processing fold 1 of 2
 + Dataset scoring 2 of 5
 - Processing fold 0 of 2
 - Processing fold 1 of 2
 + Dataset scoring 3 of 5
 - Processing fold 0 of 2
 - Processing fold 1 of 2
 + Dataset scoring 4 of 5
 - Processing fold 0 of 2
 - Processing fold 1 of 2
-750.202
